# Feature Engineering for Classification

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import datetime
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
za_data = pd.read_csv("Classification/all_files_ZAv1.csv")

In [3]:
za_data.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time'],
      dtype='object')

In [4]:
len(list(za_data.columns))

23

In [5]:
za_data["End Date Dt"] = za_data["End Date Dt"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [6]:
za_data.shape

(53686, 23)

In [7]:
za_data_test = za_data[za_data['End Date Dt'] > datetime(2022, 5, 26)]

za_data_test_first_time = za_data_test[za_data_test["trackAppearanceCount"] == 0]
za_data_test_first_time_df = za_data_test_first_time[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]


za_data_test = za_data_test[za_data_test["trackAppearanceCount"] != 0]
za_data = za_data[za_data['End Date Dt'] <= datetime(2022, 5, 26)]

In [8]:
za_data_df = za_data[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]
za_data_test_df = za_data_test[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]

In [9]:
# za_data["End Date"].unique().tolist()

In [10]:
groupby_cols = ['main_artist', 'track_name']

In [11]:
za_data_test_first_time

,trackAppearanceCount,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,...,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen,Artist and Track,rank difference,Position over Time
2,0,0,128,spotify:track:3KnHZwV1StOTvtHwpK2gK1,"031CHOPPA, Madumane, Focalistic, Xduppy, Kingtone",Usenemali,031Choppa,128,-1,1,...,2022,2022-09-08,1.662595e+09,2022-09-01,1,031CHOPPA,0,"031CHOPPA, Madumane, Focalistic, Xduppy, Kingt...",0,0
14,0,97,23,spotify:track:1MTXYvJ9TSqg9x6WPCDx2n,21 Savage,3AM on Glenwood,OVO / Republic Records,23,-1,1,...,2022,2022-11-10,1.668038e+09,2022-11-01,1,21 Savage,0,21 Savage; 3AM on Glenwood,0,0
251,0,0,38,spotify:track:0AY3sNqV7qAdAZ8j6TRVBd,"2Point1, Ntate Stunna, Nthabi Sings",Stimela,Penya Play Productions,38,-1,1,...,2022,2022-12-29,1.672272e+09,2022-12-01,1,2Point1,1,"2Point1, Ntate Stunna, Nthabi Sings; Stimela",0,0
918,0,58,10,spotify:track:6wuIh8EH2MYLI5su3FZ935,A-Reece,BAD GUY,REVENGE CLUB RECORDS,10,-1,1,...,2022,2022-10-27,1.666829e+09,2022-10-01,1,A-Reece,1,A-Reece; BAD GUY,0,0
921,0,59,19,spotify:track:2T8zFhRIoj0v4sD9YoZDW8,"A-Reece, Emtee",BIGGER THAN ME,REVENGE CLUB RECORDS,19,-1,1,...,2022,2022-10-27,1.666829e+09,2022-10-01,1,A-Reece,0,"A-Reece, Emtee; BIGGER THAN ME",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53644,0,0,95,spotify:track:4XKXphKH76W4zNpkgFS5o2,j-hope,MORE,BIGHIT MUSIC,95,-1,1,...,2022,2022-07-07,1.657152e+09,2022-07-01,1,j-hope,0,j-hope; MORE,0,0
53646,0,3,59,spotify:track:5wxYxygyHpbgv0EXZuqb9V,"j-hope, J. Cole",on the street (with J. Cole),BIGHIT MUSIC,59,-1,1,...,2023,2023-03-09,1.678320e+09,2023-03-01,1,j-hope,0,"j-hope, J. Cole; on the street (with J. Cole)",0,0
53657,0,0,165,spotify:track:4ZxL5Fi5K0nkyTmsbwPdsj,rum.gold,Call It What You Want,Leola LLC,165,-1,1,...,2022,2022-06-16,1.655338e+09,2022-06-01,1,rum.gold,0,rum.gold; Call It What You Want,0,0
53679,0,0,197,spotify:track:4XNO7bG8cMM21nfxcK3AIi,"uLazi, Thabza Tee",Gijima,Mguzuguzu Records,197,-1,1,...,2022,2022-07-28,1.658966e+09,2022-07-01,1,uLazi,0,"uLazi, Thabza Tee; Gijima",0,0


### 0. Entry Position into Chart

In [12]:
def findingtheentrypos(za_data):
    
    groupednum_songs = za_data.groupby(groupby_cols)["End Date"].agg("min")
    groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
    groupednum_songs_df.columns = groupby_cols + ["Artist Song Start Date"]
    
    groupednum_songs_df = groupednum_songs_df.drop_duplicates()
    groupednum_songs_df.shape
    
    groupednum_songs_ls = groupednum_songs_df.values.tolist()
    
    thefirstones = []
    for artsongls in groupednum_songs_ls:
        theartist = artsongls[0]
        thetrack = artsongls[1]
        thestartdate = artsongls[2]
        thefirstones.append(za_data[(za_data["main_artist"] == theartist) & (za_data["track_name"] == thetrack) & (za_data["End Date"] == thestartdate)])
        
    first_track_position_df = pd.concat(thefirstones)
    
    first_track_position_df_relevant = first_track_position_df[["main_artist", "track_name", "rank", "End Date"]]
    first_track_position_df_relevant.columns = ["main_artist", "track_name", "entry rank", "End Date"]
    
    groupednum_songs = za_data.groupby(groupby_cols)["End Date"].agg("count")
    groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
    groupednum_songs_df.columns = groupby_cols + ["Artist Song Count"]
    
    return groupednum_songs_df, first_track_position_df_relevant

### 1. Streams

In [13]:
def findtheMinMaxMeanStreams(za_data):
    groupedmax_streams = za_data.groupby(groupby_cols)["streams"].agg("max")
    groupedmax_streams_df = pd.DataFrame(groupedmax_streams).reset_index()
    groupedmax_streams_df.columns = groupby_cols + ["Max Streams per Song"]
    
    groupedmin_streams = za_data.groupby(groupby_cols)["streams"].agg("min")
    groupedmin_streams_df = pd.DataFrame(groupedmin_streams).reset_index()
    groupedmin_streams_df.columns = groupby_cols + ["Min Streams per Song"]
    
    groupedsum_streams = za_data.groupby(groupby_cols)["streams"].agg("sum")
    groupedsum_streams_df = pd.DataFrame(groupedsum_streams).reset_index()
    groupedsum_streams_df.columns = groupby_cols + ["Total Streams per Song"]

    groupedmean_streams = za_data.groupby(groupby_cols)["streams"].agg("mean")
    groupedmean_streams_df = pd.DataFrame(groupedmean_streams).reset_index()
    groupedmean_streams_df.columns = groupby_cols + ["Mean Streams per Song"]
    
    return groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df

### 2. Positions

In [14]:
def findtheMinMaxPositions(za_data):
    groupedpos_streams = za_data.groupby(groupby_cols)["rank"].agg("min")
    groupedpos_streams_df = pd.DataFrame(groupedpos_streams).reset_index()
    groupedpos_streams_df.columns = groupby_cols + ["Min Position per Song"]

    groupedposmax_streams = za_data.groupby(groupby_cols)["rank"].agg("max")
    groupedposmax_streams_df = pd.DataFrame(groupedposmax_streams).reset_index()
    groupedposmax_streams_df.columns = groupby_cols + ["Max Position per Song"]
    
    return groupedpos_streams_df, groupedposmax_streams_df

### 3. Dates

## Combine the dfs

In [15]:
def combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df):
    groupeddf = groupedmax_streams_df
    
    groupeddf["Min Streams per Song"] = groupedmin_streams_df["Min Streams per Song"]
    groupeddf["Total Streams per Song"] = groupedsum_streams_df["Total Streams per Song"]
    groupeddf["Mean Streams per Song"]= groupedmean_streams_df["Mean Streams per Song"]
    groupeddf["Min Position per Song"] = groupedpos_streams_df["Min Position per Song"]
    groupeddf["Max Position per Song"] = groupedposmax_streams_df["Max Position per Song"]
    groupeddf["Artist Song Count"] = groupednum_songs_df["Artist Song Count"]
    
    groupeddf["MaxMin Difference Position"] = groupeddf["Max Position per Song"] - groupeddf["Min Position per Song"]
    groupeddf["MaxMin Difference Streams"] = groupeddf["Max Streams per Song"] - groupeddf["Min Streams per Song"]
    
    groupeddf["MaxMin Ratio Position"] = groupeddf["Max Position per Song"] / groupeddf["Min Position per Song"]
    groupeddf["MaxMin Ratio Streams"] = groupeddf["Max Streams per Song"] / groupeddf["Min Streams per Song"]
    
    groupeddf.drop(["Max Position per Song"], inplace=True, axis=1)
    
    groupeddf_startpos = groupeddf.merge(first_track_position_df_relevant[["main_artist", "track_name", "entry rank"]], left_on=["main_artist", "track_name"], right_on=["main_artist", "track_name"], how="inner")
    
    groupeddf_startpos["main_art_track"] = groupeddf_startpos["main_artist"] + groupeddf_startpos["track_name"]
    
    groupeddf_startpos_regress = groupeddf_startpos[['main_artist', "main_art_track", 'entry rank', 'Min Position per Song']].drop_duplicates()
    
    return groupeddf, groupeddf_startpos_regress
    

### THE KMEANS

In [16]:
def thekmeansfunc(za_data_df, groupeddf, groupeddf_startpos_regress, groupby_cols, filenname):
    grouped_za_data_df = za_data_df.merge(groupeddf, left_on=["main_artist", "track_name"], right_on=["main_artist", "track_name"], how="inner")
    
    train_groupeddf = grouped_za_data_df.drop(groupby_cols, axis=1)
    
    kmeans = KMeans(n_clusters=4, random_state=0).fit(train_groupeddf)
    
    allklabels = kmeans.labels_
    
    print(len(grouped_za_data_df), len(train_groupeddf))
    
    grouped_za_data_df["Labels"] = allklabels
    
    other_cols = ["Min Streams per Song", "Total Streams per Song", "Mean Streams per Song", "Min Position per Song", "Artist Song Count", "MaxMin Ratio Position", "MaxMin Ratio Streams", "MaxMin Difference Position", "MaxMin Difference Streams"]
    
    for col in other_cols:
        grouped_za_data_df[col] = groupeddf[col]
        
    grouped_za_data_df.to_csv("grouped_za_data_df" + filenname +".csv", index=False)
    
    grouped_za_data_df["main_art_track"] = grouped_za_data_df["main_artist"] + grouped_za_data_df["track_name"]
    
    grouped_za_data_df = grouped_za_data_df.merge(groupeddf_startpos_regress, left_on=["main_art_track"], right_on=["main_art_track"], how="left")
    
    grouped_za_data_df.columns = ['main_artist', 'track_name', 'trackAppearanceCount', 'artistAppearanceCount', 'rank', 'peak_rank', 'previous_rank','weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference', 'Max Streams per Song', 'Min Streams per Song', 'Total Streams per Song', 'Mean Streams per Song', 'Min Position per Song', 'Artist Song Count','MaxMin Difference Position', 'MaxMin Difference Streams','MaxMin Ratio Position', 'MaxMin Ratio Streams', 'Labels', 'main_art_track', 'main_artist_y', 'entry rank', 'Min Position per Song_y']
    
    grouped_za_data_df.drop(['main_artist_y', 'Min Position per Song_y'], axis=1, inplace=True)
    
    labels_kmeans = grouped_za_data_df['Labels'].unique().tolist()
    
    grouped_za_data_df = grouped_za_data_df.dropna()
    
    print(grouped_za_data_df.shape)
    grouped_za_data_df = grouped_za_data_df.drop_duplicates()
    grouped_za_data_df.to_csv("grouped_za_data_df.csv", index=False)

    
    return grouped_za_data_df

# Call the functions

In [17]:
scaler = StandardScaler()

In [18]:
some_artists_of_interest = ["6LACK", "9umba", "BNXN fka Buju", "Black Coffee", "Big Zulu"]
groupednum_songs_df, first_track_position_df_relevant = findingtheentrypos(za_data)
groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df = findtheMinMaxMeanStreams(za_data)
groupedpos_streams_df, groupedposmax_streams_df = findtheMinMaxPositions(za_data)
groupeddf, groupeddf_startpos_regress = combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df)
filenname = "zaall"

In [19]:
groupby_cols = ['main_artist']

In [20]:
grouped_za_data_df = za_data_df.merge(groupeddf, left_on=["main_artist", "track_name"], right_on=["main_artist", "track_name"], how="inner")
    
# train_groupeddf = grouped_za_data_df.drop(groupby_cols, axis=1)

In [21]:
groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df = findtheMinMaxMeanStreams(grouped_za_data_df)
groupedpos_streams_df, groupedposmax_streams_df = findtheMinMaxPositions(grouped_za_data_df)


In [22]:
def combinedataframesjustartist(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df):
    groupeddf = groupedmax_streams_df
    
    groupeddf["Min Streams per Song"] = groupedmin_streams_df["Min Streams per Song"]
    groupeddf["Total Streams per Song"] = groupedsum_streams_df["Total Streams per Song"]
    groupeddf["Mean Streams per Song"]= groupedmean_streams_df["Mean Streams per Song"]
    groupeddf["Min Position per Song"] = groupedpos_streams_df["Min Position per Song"]
    groupeddf["Max Position per Song"] = groupedposmax_streams_df["Max Position per Song"]
    groupeddf["Artist Song Count"] = groupednum_songs_df["Artist Song Count"]
    
    groupeddf["MaxMin Difference Position"] = groupeddf["Max Position per Song"] - groupeddf["Min Position per Song"]
    groupeddf["MaxMin Difference Streams"] = groupeddf["Max Streams per Song"] - groupeddf["Min Streams per Song"]
    
    groupeddf["MaxMin Ratio Position"] = groupeddf["Max Position per Song"] / groupeddf["Min Position per Song"]
    groupeddf["MaxMin Ratio Streams"] = groupeddf["Max Streams per Song"] / groupeddf["Min Streams per Song"]
    
    groupeddf.drop(["Max Position per Song"], inplace=True, axis=1)
    

    return groupeddf

In [23]:
groupeddf = combinedataframesjustartist(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df)

In [24]:
groupeddf

,main_artist,Max Streams per Song,Min Streams per Song,Total Streams per Song,Mean Streams per Song,Min Position per Song,Artist Song Count,MaxMin Difference Position,MaxMin Difference Streams,MaxMin Ratio Position,MaxMin Ratio Streams
0,$NOT,23789,23789,23789,23789.000000,194,1,0,0,1.000000,1.000000
1,$uicideboy$,23341,23341,23341,23341.000000,182,1,0,0,1.000000,1.000000
2,2 Chainz,9137,3964,56641,5664.100000,78,7,119,5173,2.525641,2.304995
3,21 Savage,57808,1349,1508109,15547.515464,12,1,188,56459,16.666667,42.852483
4,24kGoldn,102605,9090,4825815,43475.810811,1,2,197,93515,198.000000,11.287679
...,...,...,...,...,...,...,...,...,...,...,...
855,grandson,16448,16448,16448,16448.000000,200,2,0,0,1.000000,1.000000
856,lil Shordie Scott,38151,23887,212781,30397.285714,73,2,84,14264,2.150685,1.597145
857,pewdiepie,5829,5829,5829,5829.000000,191,1,0,0,1.000000,1.000000
858,salem ilese,29285,16128,256035,21336.250000,53,1,133,13157,3.509434,1.815786


In [25]:
theartists = groupeddf[groupby_cols]

In [26]:
train_groupeddf = groupeddf.drop(groupby_cols, axis=1)

In [27]:
train_groupeddf.columns

Index(['Max Streams per Song', 'Min Streams per Song',
       'Total Streams per Song', 'Mean Streams per Song',
       'Min Position per Song', 'Artist Song Count',
       'MaxMin Difference Position', 'MaxMin Difference Streams',
       'MaxMin Ratio Position', 'MaxMin Ratio Streams'],
      dtype='object')

In [28]:
train_groupeddf = train_groupeddf[['Mean Streams per Song', 'Artist Song Count', 'MaxMin Ratio Streams', 'MaxMin Ratio Position', 'Min Position per Song']]

In [35]:
kmeans = KMeans(n_clusters=12, random_state=0).fit(train_groupeddf)
allklabels = kmeans.labels_

In [36]:
groupeddf["Labels"] = allklabels

In [37]:
groupeddf.to_csv("Classification/groupeddfinteresting_16May2023.csv", index=False)

In [38]:
# za_data

In [39]:
grouped_za_data_df = za_data.merge(groupeddf, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [40]:
grouped_za_data_df.to_csv("Classification/groupedza_16May2023.csv", index=False)